In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten ## 拉平
from tensorflow.keras.optimizers import SGD

我的嘗試
1. 倒數第二層dense層node數改為100、epochs=20 -> 測試準確率97.68%
2. 增加1個Conv層，filter數為128 -> 測試準確率97.39%
3. 減少第3個Conv層 -> 測試準確率96.9%
4. 把batch size調成64 -> 測試準確率97.8%

最終模型：
倒數第二層dense層node數改為100、epochs=20、batch size=64，測試準確率97.8%

### 1. 讀入 MNSIT 數據集

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255 
##若是彩色會有1會變成3

In [ ]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
# model.add(Conv2D(128, (3,3), padding='same',
#                 activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(100, activation='relu'))

In [ ]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 576)              

#### 組裝

In [ ]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs=20)

Epoch 1/20
938/938 [==============================] - 4s 3ms/step - loss: 0.0899 - accuracy: 0.1265
Epoch 2/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0885 - accuracy: 0.2461
Epoch 3/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0595 - accuracy: 0.5591
Epoch 4/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0152 - accuracy: 0.9037
Epoch 5/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0101 - accuracy: 0.9354
Epoch 6/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0077 - accuracy: 0.9513
Epoch 7/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0064 - accuracy: 0.9583
Epoch 8/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0057 - accuracy: 0.9635
Epoch 9/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0049 - accuracy: 0.9688
Epoch 10/20
938/938 [==============================] - 3s 3ms/step - loss: 0.0047 - accuracy: 0.9693

### Step 3. 預測

In [ ]:
result = model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


### 看看測試資料表現如何

In [ ]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0034 - accuracy: 0.9780


In [ ]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 97.80%


In [ ]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [ ]:
from ipywidgets import interact_manual

In [ ]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [ ]:
model.save('myCNNmodel.hw4')

INFO:tensorflow:Assets written to: myCNNmodel.hw4/assets
